In [1]:
import requests
import csv
import time
import random
from bs4 import BeautifulSoup
import datetime
import re
import json
from urllib.parse import unquote

class WeiboLabubuSpider:
    def __init__(self):
        self.session = requests.Session()
        
        # 完整的Cookie
        self.cookie_str = "SCF=ArQEioGfAJOkULRrtbDCrRG2xHkl8TQR7GJyFDOjMv3gmt9EKTQb-xazZ5XGfvfJtDnGKBo-B7J9zRuwuqkSgUc.; SUB=_2A25EJd1oDeRhGeFH6VQV9CjJzTuIHXVnW1CgrDV8PUNbmtAbLWvfkW9Ne4IpXzIJq-AmkjdTqJEIuNIeoPkr4kqZ; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WFxepMhUzOM4bmuWyIrwP6s5NHD95QN1KzcShBcSKqNWs4DqcjMi--NiK.Xi-2Ri--ciKnRi-zNS0.ESoBXSo-cS7tt; ALF=02_1766406712; WBPSESS=ioUjuU2U_k0Foa-0l2Np7wFskK9Wy9s3VdShzHRUj-kpfzYQf6IwP4e6Lbc_R6Q1h9n53n-qTyrbCXKaZwPUaKzJy0GR8SCslbYkZfJq6d7TNoa80x0y1w83ZWgBVoEhBQUf6WVptDeWBQHxc_JZRg==; SINAGLOBAL=57139976085.536224.1763817397666; XSRF-TOKEN=VtWCEf3C18OOv6bw3vqHpJI1; _s_tentry=weibo.com; Apache=386424019237.892.1763844122963; ULV=1763844122965:2:2:1:386424019237.892.1763844122963:1763817397668"
        
        # 设置更真实的请求头
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 Edg/120.0.0.0',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
            'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
            'Accept-Encoding': 'gzip, deflate, br',
            'Referer': 'https://weibo.com/',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1',
            'Sec-Fetch-Dest': 'document',
            'Sec-Fetch-Mode': 'navigate',
            'Sec-Fetch-Site': 'same-origin',
            'Sec-Fetch-User': '?1',
            'Cache-Control': 'max-age=0',
        }
        
        self._set_cookies()
        self.session.headers.update(self.headers)
        
        # 爬取统计
        self.success_pages = 0
        self.failed_pages = 0
        self.filtered_count = 0  # 记录过滤掉的微博数量
    
    def _set_cookies(self):
        """设置Cookie"""
        cookie_dict = {}
        for item in self.cookie_str.split(';'):
            item = item.strip()
            if '=' in item:
                key, value = item.split('=', 1)
                cookie_dict[key.strip()] = value.strip()
        
        requests_cookies = requests.utils.cookiejar_from_dict(cookie_dict)
        self.session.cookies.update(requests_cookies)
        print("Cookie设置完成")
    
    def verify_login(self):
        """验证登录状态"""
        try:
            # 访问个人主页验证登录状态
            response = self.session.get('https://weibo.com', timeout=10, allow_redirects=False)
            
            if response.status_code == 302:
                location = response.headers.get('Location', '')
                if 'login' in location or 'passport' in location:
                    print("Cookie无效，需要重新登录")
                    return False
            
            # 检查页面内容
            if '微博' in response.text and '登录' not in response.text:
                print("登录状态验证成功")
                return True
            else:
                print("登录状态异常")
                return False
                
        except Exception as e:
            print(f"验证登录状态失败: {e}")
            return False
    
    def is_labubu_related(self, content):
        """严格检查内容是否与labubu相关"""
        if not content:
            return False
        
        # labubu相关关键词
        labubu_keywords = [
            '#labubu#', '#Labubu#', '#LABUBU#',  # 话题标签
            'labubu', 'Labubu', 'LABUBU',        # 英文
            '拉布布', '拉布布',                    # 中文
        ]
        
        content_lower = content.lower()
        
        # 包含labubu相关关键词
        for keyword in labubu_keywords:
            if keyword.lower() in content_lower:
                return True
        
        return False
    
    def search_weibo(self, keyword, pages=50):
        """搜索微博，严格过滤只保留labubu相关内容"""
        print(f"开始搜索微博话题: {keyword}")
        print("严格过滤：只保留包含#labubu#话题的微博")
        
        weibo_data = []
        
        for page in range(1, pages + 1):
            print(f"正在获取第 {page}/{pages} 页数据...")
            
            try:
                # 使用微博搜索接口
                params = {
                    'q': keyword,
                    'type': 'all',
                    'page': page,
                    'Refer': 'g'
                }
                
                response = self.session.get(
                    'https://s.weibo.com/weibo',
                    params=params,
                    timeout=20
                )
                
                if response.status_code != 200:
                    print(f"请求失败: {response.status_code}")
                    self.failed_pages += 1
                    continue
                
                # 检查反爬
                if any(word in response.text for word in ['安全验证', '验证码', '异常访问']):
                    print("触发反爬机制，暂停爬取")
                    break
                
                # 解析页面
                soup = BeautifulSoup(response.text, 'html.parser')
                
                # 查找微博卡片
                cards = self.find_all_weibo_cards(soup)
                if not cards:
                    print("未找到微博卡片，可能页面结构有变")
                    self.failed_pages += 1
                    continue
                
                print(f"找到 {len(cards)} 个微博卡片")
                
                page_count = 0
                page_filtered = 0  # 本页过滤数量
                page_weibo_data = []
                
                for i, card in enumerate(cards):
                    try:
                        weibo_info = self.parse_weibo_card_detailed(card)
                        if weibo_info and weibo_info['内容']:
                            # 严格过滤：只保留labubu相关内容
                            if self.is_labubu_related(weibo_info['内容']):
                                page_weibo_data.append(weibo_info)
                                page_count += 1
                            else:
                                page_filtered += 1
                                # 显示被过滤的内容（前几条）
                                if page_filtered <= 3:
                                    print(f"  过滤非labubu内容: {weibo_info['内容'][:50]}...")
                            
                    except Exception as e:
                        print(f"解析卡片 {i+1} 失败: {e}")
                        continue
                
                # 更新总过滤计数
                self.filtered_count += page_filtered
                
                # 添加本页过滤后的数据到总数据
                weibo_data.extend(page_weibo_data)
                
                if page_count > 0:
                    self.success_pages += 1
                    print(f"第 {page} 页成功解析 {page_count} 条labubu相关微博")
                    print(f"本页过滤掉 {page_filtered} 条非相关微博")
                    
                    # 显示本页详细统计
                    page_reposts = sum(item['转发数'] for item in page_weibo_data)
                    page_comments = sum(item['评论数'] for item in page_weibo_data)
                    page_likes = sum(item['点赞数'] for item in page_weibo_data)
                    
                    print(f"本页统计: 转发{page_reposts} 评论{page_comments} 点赞{page_likes}")
                    
                    # 显示本页前3条数据详情
                    if page_weibo_data:
                        print("本页labubu数据示例:")
                        for j, item in enumerate(page_weibo_data[:3]):
                            print(f"  {j+1}. 用户: {item['用户']}")
                            print(f"     内容: {item['内容'][:60]}...")
                            print(f"     时间: {item['发布时间']}")
                            print(f"     互动: 转发({item['转发数']}) 评论({item['评论数']}) 点赞({item['点赞数']})")
                    else:
                        print("本页无labubu相关内容")
                        
                else:
                    self.failed_pages += 1
                    print(f"本页未解析到labubu相关微博，过滤掉 {page_filtered} 条")
                
                # 总体进度显示
                total_collected = len(weibo_data)
                progress = (page / pages) * 100
                print(f"总体进度: {page}/{pages}页 ({progress:.1f}%)")
                print(f"已收集labubu微博: {total_collected} 条，总过滤: {self.filtered_count} 条")
                
                # 智能延迟
                delay = self.calculate_delay(page)
                print(f"等待 {delay:.1f} 秒后继续...")
                time.sleep(delay)
                
            except requests.exceptions.Timeout:
                print("请求超时")
                self.failed_pages += 1
                time.sleep(10)
                continue
            except requests.exceptions.ConnectionError:
                print("连接错误，等待重试...")
                self.failed_pages += 1
                time.sleep(15)
                continue
            except Exception as e:
                print(f"第 {page} 页获取失败: {e}")
                self.failed_pages += 1
                continue
        
        return weibo_data
    
    def find_all_weibo_cards(self, soup):
        """查找所有微博卡片"""
        cards = []
        
        # 主选择器
        selectors = [
            'div[action-type="feed_list_item"]',
            '.card-wrap',
            '.card',
            '.wb-card',
            '[mid]',
            'div[node-type="feed_list"]',
            '.feed_list'
        ]
        
        for selector in selectors:
            found_cards = soup.select(selector)
            if found_cards:
                cards.extend(found_cards)
        
        # 去重
        unique_cards = []
        seen_ids = set()
        for card in cards:
            card_id = self.get_card_id(card)
            if card_id and card_id not in seen_ids:
                unique_cards.append(card)
                seen_ids.add(card_id)
        
        return unique_cards
    
    def get_card_id(self, card):
        """获取卡片唯一标识"""
        # 尝试获取mid
        mid = card.get('mid') or card.get('data-mid')
        if mid:
            return f"mid_{mid}"
        
        # 尝试获取其他唯一标识
        for attr in ['id', 'data-id', 'comment_id']:
            attr_value = card.get(attr)
            if attr_value:
                return f"{attr}_{attr_value}"
        
        # 使用内容哈希作为备用标识
        content = card.get_text()[:100]
        return f"hash_{hash(content)}"
    
    def parse_weibo_card_detailed(self, card):
        """详细解析微博卡片"""
        try:
            # 用户信息
            user = self.extract_user_info(card)
            
            # 微博内容
            content = self.extract_content_detailed(card)
            
            # 发布时间
            post_time = self.extract_post_time(card)
            
            # 互动数据
            reposts, comments, likes = self.extract_interaction_detailed(card)
            
            # 使用中文键名
            weibo_info = {
                '用户': user or '未知用户',
                '内容': content or '',
                '发布时间': post_time or '',
                '转发数': reposts,
                '评论数': comments,
                '点赞数': likes,
            }
            
            return weibo_info
            
        except Exception as e:
            print(f"解析微博卡片详细失败: {e}")
            return None
    
    def extract_user_info(self, card):
        """提取用户信息"""
        user_selectors = [
            '.name', '.username', '.txt[nick-name]', '[nick-name]',
            '.sourcename', '.usertxt', '.user-name',
            'a[nick-name]', '.user_link'
        ]
        
        for selector in user_selectors:
            element = card.select_one(selector)
            if element and element.get_text().strip():
                return element.get_text().strip()
        
        return "未知用户"
    
    def extract_content_detailed(self, card):
        """详细提取微博内容"""
        content_selectors = [
            '.txt', '[node-type="feed_list_content"]', '.content', '.weibo-text',
            '.detail', '.feed_content', '.text', '.blog'
        ]
        
        for selector in content_selectors:
            element = card.select_one(selector)
            if element:
                # 清理内容
                text = element.get_text().strip()
                text = re.sub(r'\s+', ' ', text)
                text = re.sub(r'全文', '', text)
                return text
        
        return ""
    
    def extract_post_time(self, card):
        """提取发布时间"""
        time_selectors = [
            '.from', '.time', '.date', '.publish_time',
            '.create_time', '.timestamp'
        ]
        
        for selector in time_selectors:
            element = card.select_one(selector)
            if element:
                text = element.get_text().strip()
                text = re.sub(r'来自.*', '', text)
                text = re.sub(r'发布于', '', text)
                return text.strip()
        
        return ""
    
    def extract_interaction_detailed(self, card):
        """详细提取互动数据"""
        # 查找互动工具栏
        toolbar_selectors = [
            '.card-act', '.toolbar', '.feed_action',
            '.interact', '.operation', '.action'
        ]
        
        for selector in toolbar_selectors:
            toolbar = card.select_one(selector)
            if toolbar:
                return self.parse_interaction_numbers(toolbar)
        
        return 0, 0, 0
    
    def parse_interaction_numbers(self, toolbar):
        """解析互动数字"""
        number_elements = toolbar.find_all(text=re.compile(r'[\d万亿]'))
        
        reposts = comments = likes = 0
        
        for i, elem in enumerate(number_elements[:10]):
            text = elem.strip()
            count = self.parse_count(text)
            
            if count > 0:
                parent_text = elem.parent.get_text() if elem.parent else ""
                if any(word in parent_text for word in ['转发', '转发数']):
                    reposts = count
                elif any(word in parent_text for word in ['评论', '评论数']):
                    comments = count
                elif any(word in parent_text for word in ['赞', '点赞', '喜欢']):
                    likes = count
                else:
                    if i % 3 == 0:
                        reposts = count
                    elif i % 3 == 1:
                        comments = count
                    else:
                        likes = count
        
        return reposts, comments, likes
    
    def parse_count(self, text):
        """解析数字"""
        if not text or text in ['转发', '评论', '赞', '收藏', '分享']:
            return 0
        
        try:
            match = re.search(r'(\d+\.?\d*)([万亿]?)', text)
            if match:
                number = float(match.group(1))
                unit = match.group(2)
                
                if unit == '万':
                    return int(number * 10000)
                elif unit == '亿':
                    return int(number * 100000000)
                else:
                    return int(number)
            return 0
        except:
            return 0
    
    def calculate_delay(self, current_page):
        """计算延迟时间"""
        base_delay = 3
        if current_page % 5 == 0:
            base_delay += 2
        return random.uniform(base_delay, base_delay + 3)
    
    def save_to_csv(self, data, filename):
        """保存到CSV文件"""
        if not data:
            print("没有数据可保存")
            return False
        
        try:
            with open(filename, 'w', newline='', encoding='utf-8-sig') as f:
                fieldnames = ['用户', '内容', '发布时间', '转发数', '评论数', '点赞数']
                writer = csv.DictWriter(f, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            
            print(f"数据已保存到: {filename}")
            return True
        except Exception as e:
            print(f"保存失败: {e}")
            return False

def main():
    """主函数"""
    print("=" * 60)
    print("微博Labubu话题爬虫启动")
    print("严格模式：只获取包含#labubu#话题的微博")
    print("=" * 60)
    
    # 创建爬虫
    spider = WeiboLabubuSpider()
    
    # 验证登录
    if not spider.verify_login():
        print("登录验证失败，程序退出")
        return
    
    # 开始爬取
    start_time = time.time()
    
    try:
        # 搜索话题
        keyword = "#labubu#"
        pages = 50
        
        print(f"爬取目标: {keyword}")
        print(f"计划爬取页数: {pages}页")
        print(f"开始时间: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print("-" * 60)
        
        data = spider.search_weibo(keyword, pages=pages)
        
        # 最终统计
        end_time = time.time()
        total_time = end_time - start_time
        
        if data:
            # 保存数据
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"微博_纯labubu话题_{pages}页_{timestamp}.csv"
            spider.save_to_csv(data, filename)
            
            # 详细统计
            total_reposts = sum(item['转发数'] for item in data)
            total_comments = sum(item['评论数'] for item in data)
            total_likes = sum(item['点赞数'] for item in data)
            
            print(f"\n" + "="*60)
            print("最终爬取统计:")
            print(f"总页数: {pages}页")
            print(f"成功页数: {spider.success_pages}页")
            print(f"失败页数: {spider.failed_pages}页")
            print(f"成功率: {(spider.success_pages/pages*100):.1f}%")
            print(f"总微博数: {len(data)} 条 (纯labubu内容)")
            print(f"总过滤数: {spider.filtered_count} 条 (非labubu内容)")
            print(f"有效比例: {(len(data)/(len(data)+spider.filtered_count)*100):.1f}%")
            print(f"总转发数: {total_reposts}")
            print(f"总评论数: {total_comments}")
            print(f"总点赞数: {total_likes}")
            print(f"总耗时: {total_time/60:.2f} 分钟")
            print(f"平均速度: {len(data)/max(1, total_time/60):.1f} 条/分钟")
            
            # 样本展示
            print(f"\n纯labubu数据样本预览 (前5条):")
            for i, item in enumerate(data[:5]):
                print(f"{i+1}. 用户: {item['用户']}")
                print(f"   内容: {item['内容'][:80]}...")
                print(f"   时间: {item['发布时间']}")
                print(f"   互动: 转发({item['转发数']}) 评论({item['评论数']}) 点赞({item['点赞数']})")
                print()
                
        else:
            print("未获取到任何labubu相关数据")
            
    except KeyboardInterrupt:
        print("用户中断爬取")
    except Exception as e:
        print(f"爬取过程出错: {e}")
        
    finally:
        print(f"\n爬虫执行完成!")
        print("=" * 60)

if __name__ == "__main__":
    main()

微博Labubu话题爬虫启动
严格模式：只获取包含#labubu#话题的微博
Cookie设置完成
登录状态验证成功
爬取目标: #labubu#
计划爬取页数: 50页
开始时间: 2025-11-26 15:35:38
------------------------------------------------------------
开始搜索微博话题: #labubu#
严格过滤：只保留包含#labubu#话题的微博
正在获取第 1/50 页数据...
找到 49 个微博卡片
第 1 页成功解析 42 条labubu相关微博
本页过滤掉 0 条非相关微博
本页统计: 转发10898 评论426 点赞1526
本页labubu数据示例:
  1. 用户: POPMART泡泡玛特
     内容: 现在是凌晨一点，#LABUBU#拍了拍你...并邀请你再熬一夜！🔜#THEMONSTERS凌晨一点系列# POPMAR...
     时间: 11月11日 18:30
     互动: 转发(136) 评论(21) 点赞(22)
  2. 用户: 在圈里收瓜
     内容: 我嘞个豆！这么可爱的迷你拉布布到底是谁在拥有！这个萌态真的无人能敌#labubu# ​...
     时间: 08月28日 23:37
     互动: 转发(474) 评论(22) 点赞(123)
  3. 用户: POPMART泡泡玛特
     内容: POPMART泡泡玛特超话 #LABUBU# ✉️致每一位深夜未眠的灵魂：你还在辗转反侧吗？当指针指向凌晨一点，LAB...
     时间: 11月07日 18:00
     互动: 转发(119) 评论(36) 点赞(47)
总体进度: 1/50页 (2.0%)
已收集labubu微博: 42 条，总过滤: 0 条
等待 4.1 秒后继续...


/var/folders/6x/flvt6fj16y72x3q31myq2m1w0000gn/T/ipykernel_31849/3009899185.py:376: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  number_elements = toolbar.find_all(text=re.compile(r'[\d万亿]'))


正在获取第 2/50 页数据...
找到 22 个微博卡片
第 2 页成功解析 20 条labubu相关微博
本页过滤掉 0 条非相关微博
本页统计: 转发90 评论26 点赞68
本页labubu数据示例:
  1. 用户: 与天堂相遇
     内容: #labubu# O绿洲 ​...
     时间: 11月24日 13:44
     互动: 转发(0) 评论(0) 点赞(0)
  2. 用户: sunshine飛625
     内容: 18块钱入手一个labubu紫色挂件，真的很便宜，不亏，后面是那种偏迷彩色，前面颜色是我最爱的紫色，好好看，如果在杭州买...
     时间: 11月24日 12:46
     互动: 转发(0) 评论(1) 点赞(0)
  3. 用户: POPMART泡泡玛特
     内容: POPMART泡泡玛特超话 机器人商店携手THE MONSTERS 凌晨一点主题惊喜上线，解锁属于深夜的奇趣小世界！🌙...
     时间: 11月23日 18:26 转赞人数超过30
     互动: 转发(45) 评论(2) 点赞(32)
总体进度: 2/50页 (4.0%)
已收集labubu微博: 62 条，总过滤: 0 条
等待 3.4 秒后继续...
正在获取第 3/50 页数据...
找到 22 个微博卡片
第 3 页成功解析 20 条labubu相关微博
本页过滤掉 0 条非相关微博
本页统计: 转发48 评论22 点赞14
本页labubu数据示例:
  1. 用户: 恩心酱-Queenie
     内容: 齐活~喜欢的东西，值得一切奔跑#泡泡玛特##labubu##盲盒##潮玩# ​...
     时间: 11月22日 20:44
     互动: 转发(0) 评论(1) 点赞(1)
  2. 用户: 罗铭郡
     内容: 水果姐收到了专属labubuΣ（ﾟдﾟlll）一脸嫌弃的丢掉了哈哈哈哈#水果姐##labubu# ​...
     时间: 11月22日 20:36
     互动: 转发(0) 评论(1) 点赞(0)
  3. 用户: 威虎新闻
     内容: 买到瑕疵#labubu#售后难#泡泡玛特被告了#日前，消费者洪先生买到了一款有瑕疵的“Labubu”玩偶，将泡泡玛特公司...
     时